# 1) Imports

In [1]:
from json import loads, JSONDecodeError
from os.path import exists
from re import match
from requests import get
from urllib.parse import urlparse

import pandas as pd
# Para poder usar o pd.read_excel
# !pip list | findstr 'xlrd'

# 2) Funções Auxiliares

## 2.1) Obter dados da API
- [Link do repositório](https://github.com/fredrike/googlescholar-api)

In [2]:
def fetch_google_scholar_api(user_id: str) -> dict:
	endpoint = f'http://cse.bth.se/~fer/googlescholar-api/googlescholar.php?user={user_id}'
	page = get(endpoint)

	if not page.ok:
		raise Exception('Erro ao completar requisição para API')

	try:
		result = loads(page.text)
		return result
		
	except JSONDecodeError as json_err:
		print(json_err)
		raise Exception('Erro ao transformar conteúdo de página para dicionário')

	except Exception as err:
		print(err)
		raise Exception('Erro ao obter dados da API')

## 2.2) Obter id a partir do link de uma página de pesquisador

In [3]:
def get_researcher_id(profile_url: str) -> str:
	result = urlparse(profile_url)
	if not result.query:
		raise Exception('Sem query')

	query = result.query
	pattern = "user=[0-9a-zA-Z]+"
	has_user_id = match(pattern, query)

	if not has_user_id:
		raise Exception('Sem id do usuário')
	
	user_id = has_user_id.group().split('=')[1]
	return user_id

## 2.3) Obter nº de citações do último ano

In [4]:
def get_latest_year_citation(user_info: dict) -> int:
	_, citations = [*user_info['citations_per_year'].items()][-1]
	return citations

## 2.4) Obter planilha como DataFrame

In [5]:
def load_spreadsheet(file_path: str) -> pd.DataFrame:
	if not exists(file_path):
		raise Exception("Planilha não existe")
	
	df_spreadsheet = pd.read_excel(file_path)
	df_filtered_spreadsheet = df_spreadsheet.dropna(axis=1)

	return df_filtered_spreadsheet

# 3) Execução

In [6]:
planilha = 'sample.xls'
df_planilha = load_spreadsheet(planilha)
df_planilha

,Pesquisador;
0,https://scholar.google.com/citations?user=ztd9...
1,https://scholar.google.com/citations?user=dxEW...
2,https://scholar.google.com/citations?user=UX9G...


In [7]:
def get_citations(researcher_url: str) -> int:
	user_id = get_researcher_id(researcher_url)
	user_info = fetch_google_scholar_api(user_id)
	citations = get_latest_year_citation(user_info)
	return citations

In [8]:
df_planilha['Citações'] = df_planilha['Pesquisador;'].apply(get_citations)
df_planilha

,Pesquisador;,Citações
0,https://scholar.google.com/citations?user=ztd9...,6
1,https://scholar.google.com/citations?user=dxEW...,7
2,https://scholar.google.com/citations?user=UX9G...,81
